Problem 9 - circuit simplification

In [ ]:
import numpy as np
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, transpile
import math 

import re
import networkx as nx

path = "C:/Users/elena/OneDrive/IQuhack/P9_grand_summit.qasm"

qc = QuantumCircuit.from_qasm_file(path)
qasm_text = open(path).read()
qasm_file = path

In [ ]:
# ---------- helpers ----------
TWOPI = 2 * math.pi

def wrap_angle(theta: float) -> float:
    """Wrap to (-pi, pi] and drop tiny angles."""
    theta = theta % TWOPI
    if theta > math.pi:
        theta -= TWOPI
    if abs(theta) < 1e-12:
        theta = 0.0
    return theta

def to_float(x):
    """Convert QASM numeric params safely."""
    try:
        return float(x)
    except Exception as e:
        raise TypeError(
            f"Gate parameter isn't a plain number: {x} ({type(x)}). "
            "If this happens, tell me and we’ll handle ParameterExpressions."
        ) from e

In [ ]:
# ---------- Step 1: Decompose u3 -> rz ry rz (keep cz) ----------
def manual_u3_to_rz_ry_rz(qc: QuantumCircuit) -> QuantumCircuit:
    """
    Convert a circuit containing u3/cz (and possibly barriers/measure)
    into a circuit over {rz, ry, cz, barrier, measure}.

    u3(theta, phi, lam) -> rz(phi); ry(theta); rz(lam)
    """
    out = QuantumCircuit(qc.num_qubits, qc.num_clbits)

    for inst, qargs, cargs in qc.data:
        name = inst.name
        qs = [q._index for q in qargs]

        if name == "u3":
            theta, phi, lam = map(to_float, inst.params)
            out.rz(phi, qs[0])
            out.ry(theta, qs[0])
            out.rz(lam, qs[0])

        elif name == "cz":
            out.cz(qs[0], qs[1])

        elif name == "barrier":
            out.barrier(*qs)

        elif name == "measure":
            # qargs: 1 qubit, cargs: 1 clbit
            out.measure(qargs[0], cargs[0])

        elif name == "reset":
            out.reset(qargs[0])

        else:
            # If the QASM truly only has u3 and cz, this won't trigger.
            # But leaving this makes the code robust.
            out.append(inst, qargs, cargs)

    return out

In [ ]:
# ---------- Step 2: Merge/push RZ through CZ ----------
def merge_rz_push_through_cz(qc: QuantumCircuit) -> QuantumCircuit:
    """
    Sweep and accumulate RZ per qubit.
    RZ commutes with CZ, so we can push across CZ and merge.
    We must flush before any non-commuting op on that qubit (e.g. RY, measure).
    """
    n = qc.num_qubits
    out = QuantumCircuit(n, qc.num_clbits)
    pending = [0.0] * n

    def flush(qi: int):
        ang = wrap_angle(pending[qi])
        if ang != 0.0:
            out.rz(ang, qi)
        pending[qi] = 0.0

    def flush_many(qubits):
        for qi in qubits:
            flush(qi)

    def flush_all():
        for qi in range(n):
            flush(qi)

    for inst, qargs, cargs in qc.data:
        name = inst.name
        qs = [q._index for q in qargs]

        if name == "rz":
            pending[qs[0]] += to_float(inst.params[0])
            continue

        if name == "cz":
            # commutes with pending RZ on both qubits
            out.cz(qs[0], qs[1])
            continue

        if name == "ry":
            flush(qs[0])
            out.ry(to_float(inst.params[0]), qs[0])
            continue

        if name == "barrier":
            # barrier is a "stop sign" for reordering: flush on involved qubits
            flush_many(qs)
            out.barrier(*qs)
            continue

        if name == "measure":
            flush(qs[0])
            out.measure(qargs[0], cargs[0])
            continue

        if name == "reset":
            flush(qs[0])
            out.reset(qargs[0])
            continue

        # conservative fallback
        flush_many(qs)
        out.append(inst, qargs, cargs)

    flush_all()
    return out


In [ ]:
from qiskit.qasm2 import dumps
# ---------- Pipeline ----------
def simplify_qasm_circuit(qasm_path: str, save_path: str = None):
    qc = QuantumCircuit.from_qasm_file(qasm_path)
    print("Loaded:", qasm_path)
    print("Original ops:", qc.count_ops())

    qc_basis = manual_u3_to_rz_ry_rz(qc)
    print("After u3->rzryrz:", qc_basis.count_ops())

    qc_simpl = merge_rz_push_through_cz(qc_basis)
    print("After RZ merge/push:", qc_simpl.count_ops())

    if save_path is not None:
        with open(save_path, "w") as f:
            f.write(dumps(qc_simpl))
        print("Saved simplified QASM to:", save_path)

    return qc, qc_basis, qc_simpl

In [ ]:
qasm_path = r"C:/Users/elena/OneDrive/IQuhack/P9_grand_summit.qasm"
qc, qc_basis, qc_simpl = simplify_qasm_circuit(
    qasm_path,
    save_path=r"C:/Users/elena/OneDrive/IQuhack/P9_simplified_rz_ry_cz.qasm"
) #this just calls the simplify function in which we loaded the original circuit, 
#decomposed the u3 gates into rzryrz gates usinf the decompose function, 
#and then merged the rz using the merge function
#the simplify function outputs the 
#qc: original circuit 
#qc_basis: circuit after u3 decomposition
#qc_simpl: circuit after rz merging

#the save_path  saves the simplified new circuit into a new qasm file called P9_simplified_rz_ry_cz.qasm